In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Define the data directory and class names
data_dir = 'data'
disease_types = ['tumor', 'no_tumor']

In [3]:
# Load the data
train_data = []
for defects_id, sp in enumerate(disease_types):
    for file in os.listdir(os.path.join(data_dir, sp)):
        train_data.append(['{}/{}'.format(sp, file), defects_id, sp])

train = pd.DataFrame(train_data, columns=['File', 'DiseaseID', 'DiseaseType'])
train = train.sample(frac=1, random_state=42)  # Shuffle and reset indices
train.index = np.arange(len(train))

In [11]:
# Parameters
IMAGE_SIZE = 128  # EfficientNet requires images of size 224x224
BATCH_SIZE = 64
EPOCHS = 25

In [12]:
# Function to read and preprocess images
def read_image(filepath):
    return cv2.imread(os.path.join(data_dir, filepath))

def resize_image(image, image_size):
    return cv2.resize(image.copy(), image_size, interpolation=cv2.INTER_AREA)

In [13]:
# Load and preprocess images
X_train = np.zeros((train.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3))
for i, file in tqdm(enumerate(train['File'].values)):
    image = read_image(file)
    if image is not None:
        X_train[i] = resize_image(image, (IMAGE_SIZE, IMAGE_SIZE))

3123it [00:20, 152.75it/s]


In [14]:
X_train = X_train.astype(np.float32) / 255.0

# Prepare labels
Y_train = train['DiseaseID'].values
Y_train = to_categorical(Y_train, num_classes=2)

# Split data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [15]:
# Build the EfficientNet model
def build_efficientnet():
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

    # Create custom model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    # Add the output layer
    output = Dense(2, activation='softmax')(x)
    
    # Create the final model
    model = Model(inputs=base_model.input, outputs=output)
    
    # Compile the model
    optimizer = Adam(learning_rate=0.002)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    
    return model

In [16]:
# Instantiate the model
model = build_efficientnet()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 128, 128, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 128, 128, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 128, 128, 3)          0         ['normalization[0][0]']   

 y)                                                                  'block2a_se_expand[0][0]']   
                                                                                                  
 block2a_project_conv (Conv  (None, 32, 32, 24)           2304      ['block2a_se_excite[0][0]']   
 2D)                                                                                              
                                                                                                  
 block2a_project_bn (BatchN  (None, 32, 32, 24)           96        ['block2a_project_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 block2b_expand_conv (Conv2  (None, 32, 32, 144)          3456      ['block2a_project_bn[0][0]']  
 D)                                                                                               
          

                                                                                                  
 block3a_project_conv (Conv  (None, 16, 16, 40)           5760      ['block3a_se_excite[0][0]']   
 2D)                                                                                              
                                                                                                  
 block3a_project_bn (BatchN  (None, 16, 16, 40)           160       ['block3a_project_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 block3b_expand_conv (Conv2  (None, 16, 16, 240)          9600      ['block3a_project_bn[0][0]']  
 D)                                                                                               
                                                                                                  
 block3b_e

 block4a_project_conv (Conv  (None, 8, 8, 80)             19200     ['block4a_se_excite[0][0]']   
 2D)                                                                                              
                                                                                                  
 block4a_project_bn (BatchN  (None, 8, 8, 80)             320       ['block4a_project_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 block4b_expand_conv (Conv2  (None, 8, 8, 480)            38400     ['block4a_project_bn[0][0]']  
 D)                                                                                               
                                                                                                  
 block4b_expand_bn (BatchNo  (None, 8, 8, 480)            1920      ['block4b_expand_conv[0][0]'] 
 rmalizati

 ormalization)                                                                                    
                                                                                                  
 block4c_drop (Dropout)      (None, 8, 8, 80)             0         ['block4c_project_bn[0][0]']  
                                                                                                  
 block4c_add (Add)           (None, 8, 8, 80)             0         ['block4c_drop[0][0]',        
                                                                     'block4b_add[0][0]']         
                                                                                                  
 block5a_expand_conv (Conv2  (None, 8, 8, 480)            38400     ['block4c_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block5a_e

 block5b_add (Add)           (None, 8, 8, 112)            0         ['block5b_drop[0][0]',        
                                                                     'block5a_project_bn[0][0]']  
                                                                                                  
 block5c_expand_conv (Conv2  (None, 8, 8, 672)            75264     ['block5b_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block5c_expand_bn (BatchNo  (None, 8, 8, 672)            2688      ['block5c_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block5c_expand_activation   (None, 8, 8, 672)            0         ['block5c_expand_bn[0][0]']   
 (Activati

 ormalization)                                                                                    
                                                                                                  
 block6b_expand_conv (Conv2  (None, 4, 4, 1152)           221184    ['block6a_project_bn[0][0]']  
 D)                                                                                               
                                                                                                  
 block6b_expand_bn (BatchNo  (None, 4, 4, 1152)           4608      ['block6b_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block6b_expand_activation   (None, 4, 4, 1152)           0         ['block6b_expand_bn[0][0]']   
 (Activation)                                                                                     
          

 block6c_add (Add)           (None, 4, 4, 192)            0         ['block6c_drop[0][0]',        
                                                                     'block6b_add[0][0]']         
                                                                                                  
 block6d_expand_conv (Conv2  (None, 4, 4, 1152)           221184    ['block6c_add[0][0]']         
 D)                                                                                               
                                                                                                  
 block6d_expand_bn (BatchNo  (None, 4, 4, 1152)           4608      ['block6d_expand_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 block6d_expand_activation   (None, 4, 4, 1152)           0         ['block6d_expand_bn[0][0]']   
 (Activati

                                                                                                  
 top_bn (BatchNormalization  (None, 4, 4, 1280)           5120      ['top_conv[0][0]']            
 )                                                                                                
                                                                                                  
 top_activation (Activation  (None, 4, 4, 1280)           0         ['top_bn[0][0]']              
 )                                                                                                
                                                                                                  
 global_average_pooling2d (  (None, 1280)                 0         ['top_activation[0][0]']      
 GlobalAveragePooling2D)                                                                          
                                                                                                  
 dropout (

In [19]:
# Define callbacks
checkpoint = ModelCheckpoint(
    filepath='efficientnet_model_brain_tumor.h5',  # Save the model as an HDF5 file
    monitor='val_loss',
    save_best_only=True,
    verbose=1,
    mode='min'
)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
annealer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, min_lr=1e-4)

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(rotation_range=360, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True, vertical_flip=True)
datagen.fit(X_train)
# Train the model
hist = model.fit(datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE), steps_per_epoch=X_train.shape[0] // BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, Y_val), verbose=2, callbacks=[checkpoint, annealer, early_stopping])

Epoch 1/25

Epoch 1: val_loss improved from inf to 3.01790, saving model to efficientnet_model_brain_tumor.h5


C:\Users\addur\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


39/39 - 165s - loss: 0.2195 - accuracy: 0.9191 - val_loss: 3.0179 - val_accuracy: 0.1536 - lr: 0.0020 - 165s/epoch - 4s/step
Epoch 2/25

Epoch 2: val_loss did not improve from 3.01790
39/39 - 148s - loss: 0.1697 - accuracy: 0.9363 - val_loss: 3.6760 - val_accuracy: 0.1536 - lr: 0.0020 - 148s/epoch - 4s/step
Epoch 3/25

Epoch 3: val_loss did not improve from 3.01790
39/39 - 139s - loss: 0.1267 - accuracy: 0.9540 - val_loss: 3.6288 - val_accuracy: 0.1536 - lr: 0.0020 - 139s/epoch - 4s/step
Epoch 4/25

Epoch 4: val_loss did not improve from 3.01790
39/39 - 123s - loss: 0.1231 - accuracy: 0.9569 - val_loss: 6.6688 - val_accuracy: 0.1536 - lr: 0.0020 - 123s/epoch - 3s/step
Epoch 5/25

Epoch 5: val_loss did not improve from 3.01790
39/39 - 122s - loss: 0.1052 - accuracy: 0.9601 - val_loss: 12.7726 - val_accuracy: 0.1536 - lr: 0.0020 - 122s/epoch - 3s/step
Epoch 6/25

Epoch 6: val_loss did not improve from 3.01790

Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0010000000474974513.
39